### Import packages

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import TimeoutException
import joblib

In [2]:
_2b_link = "https://2b.com.eg/en/computers/laptops.html?p=1"
links = []

# open chromedriver on your PC

def open_chromedriver():

    from selenium.webdriver import Chrome, ChromeOptions # Pictures are not displayed in page
                                                         # To download components quickly
        
    from selenium.webdriver.chrome.service import Service # to install The appropriate version for the user's browser
        
    from webdriver_manager.chrome import ChromeDriverManager 
    
    chromedriver_installer = Service(ChromeDriverManager().install()) 
    
    chromedriver_options = ChromeOptions()
    prefs = {'profile.managed_default_content_settings.images' : 2} # Pictures are not displayed in page
    chromedriver_options.add_experimental_option('prefs', prefs)
    
    web = Chrome(service = chromedriver_installer, chrome_options = chromedriver_options)
    
    return web


# scroll down all the page to load all data of item                       
def scroll_down_all_the_way():
    
    from time import sleep
    
    global scroll_delay_s
    
    end = 0
    while end < web.execute_script('return document.body.scrollHeight'):
        start = end
        end = start + 1000
        web.execute_script(f'window.scrollTo({start}, {end})')
        sleep(2)

    
# get all links of laptops    
def get_laptops_links():
    global page_num
    for page in range(1, page_num+1):
        website_link = f"https://2b.com.eg/en/computers/laptops.html?p={page}"
        web.get(website_link) # open webdriver
        scroll_down_all_the_way()
        
        global soup
        soup = BeautifulSoup(web.page_source, 'html.parser') # send request

        tag = soup.find("div", id="layer-product-list")
        global links

        info = tag.find("ol", class_="filterproducts products list items product-items").find_all("li")
        links = [link.a["href"] for link in info]
    
    return links

# scrape data and append it in list of dictionary
def get_laptops_info(link):
    web.get(link)
    listing = {}
    listing["link"] = link
    time.sleep(3)
    global soup
    soup = BeautifulSoup(web.page_source, 'html.parser')
    info = soup.find("div", class_="product-info-main") # content of page
    
    if info:
        listing["title"] = info.find("div", class_="page-title-wrapper product").get_text(strip=True)
        listing["price"] = info.find("div", class_="price-box price-final_price").get_text(strip=True).split("Price")[-1]
        
        global l1
        global l2
        global l
        
        l1 = []
        l2 = []
        l = []
        
        desc = info.p.get_text(strip=True).split(":")
        for i in range(len(desc)):
            if i %2 == 0:
                l1.append(desc[i])

            if i %2 != 0:
                l2.append(desc[i])

        for defn, value in zip(l1, l2):
            l.append(f"{defn} : {value}")
            
        listing['more_info'] = "  ;  ".join(l) # Concate all features and spliting it by ";" 
        
    try:
        
        add_data = web.find_element(by="id", value="tab-label-additional").click()
    except:
        try:
            time.sleep(2)
            add_data = web.find_element(by="id", value="tab-label-additional").click()
        except:
            time.sleep(3)
            add_data = web.find_element(by="id", value="tab-label-additional").click()
    
    feature = soup.find("div", id="additional")
     # to save data in dectionary   
    if feature:                    
        global Keys
        global values

        tag = feature.find_all("tr")
        keys = [x.find_all("th")[0].get_text(strip=True) for x in tag]
        values = [x.find_all("td")[0].get_text(strip=True) for x in tag]
        dict_value = dict(zip(keys, values))
        listing.update(dict_value)

    print(link)
    print(f"{len(listing)} - ", listing)
    print("-" *100, "\n\n")
    

    return listing



# to save all operation on PC
def save_state():
    
    import joblib
    
    global listings
    global state_filepath
    
    state = listings
    joblib.dump(state, state_filepath)

In [ ]:
state_filepath = r"E:\other\Epsilon_Ai\Final Project\final project\2_B.pkl"

listings = []
web = open_chromedriver()
web.get(_2b_link)  
scroll_down_all_the_way()
# to get all pages of website
page_num = int(web.find_element(by="xpath", value="/html/body/div[4]/main/div[4]/div[1]/div[3]/div[3]/div[3]/ul/li[4]/a/span[2]").text)
links = get_laptops_links()


while links: # to git all links
    link = links.pop() # to search about link by link
    try:
        listing = get_laptops_info(link)
        listings.append(listing)
                
    except :
        print('timeout detected... cooling off...') # to make sure all informations is loaded
        time.sleep(60.0)
        listing = get_laptops_info(link)
        listings.append(listing)


save_state()



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\Mora\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache
C:\Users\Mora\AppData\Local\Temp/ipykernel_19676/2099593986.py:21: DeprecationWarning: use options instead of chrome_options
  web = Chrome(service = chromedriver_installer, chrome_options = chromedriver_options)


https://2b.com.eg/en/huawei-matebook-e-intelr-coretm-i7-1160g7-16gb-512gb-ssd-intelr-irisr-xe-graphics-12-6-oled-win11-nebula-gray.html
24 -  {'link': 'https://2b.com.eg/en/huawei-matebook-e-intelr-coretm-i7-1160g7-16gb-512gb-ssd-intelr-irisr-xe-graphics-12-6-oled-win11-nebula-gray.html', 'title': 'Huawei Matebook E - Intel® Core™ i7-1160G7 - 16GB - 512GB SSD - Intel® Iris® Xe Graphics - 12.6" OLED - Win11 - Nebula Gray', 'price': 'EGP32,499.00', 'more_info': 'Processor : 11th Generation Intel® Core™ i7-1160G7 Processor 12M Cache, up to 4.40 GHz, with IPURAM  ;  16GBHard Disk : 512GB\xa0SSDGraphics Card  ;  Intel® Iris® Xe GraphicsColor : Nebula Gray', 'Brand': 'HUAWEI', 'Color': 'Gray', 'Product Type': 'Laptop', 'Part Number': '53012SWP', 'Model': 'Huawei Matebook E - DRC-W76', 'Processor': 'Intel Core i7', 'Processor Generation': '11th Generation', 'Processor Information': '11th Generation Intel® Core™ i7-1160G7 Processor 12M Cache, up to 4.40 GHz, with IPU', 'RAM': '16 GB', 'RAM Inf

In [3]:
listings = joblib.load(r"E:\other\Epsilon_Ai\Final Project\final project\2_B.pkl") # load data form PC
listings

[{'link': 'https://2b.com.eg/en/acer-nitro-5-an515-57-743y-intelr-coretm-i7-11800h-16gb-1tb-ssd-nvidiar-geforcer-rtx-3050-4gb-15-6-fhd-black.html',
  'title': 'Acer Nitro 5 AN515-57-743Y - Intel® Core™ i7-11800H - 16GB - 1TB SSD - NVIDIA® GeForce® RTX 3050 4GB - 15.6" FHD - Black',
  'price': 'EGP22,999.00',
  'more_info': 'Processor : 11th Generation Intel® Core™ i7-11800H Processor 24M Cache, up to 4.60 GHzRam  ;  16GBHard Disk : 1TB SSDGraphics Card  ;  NVIDIA® GeForce® RTX 3050 4GBColor : Black',
  'Brand': 'Acer',
  'Color': 'Black',
  'Product Type': 'Laptop',
  'Part Number': 'NH.QELEM.006',
  'Model': 'Acer Nitro 5 AN515-57-743Y',
  'Processor': 'Intel Core i7',
  'Processor Generation': '11th Generation',
  'Processor Information': '11th Generation Intel® Core™ i7-11800H Processor 24M Cache, up to 4.60 GHz',
  'RAM': '16 GB',
  'RAM Information': '16GB DDR4 3200MHz',
  'Hard Disk Capacity': '1 Tera',
  'Hard Disk Type': '1TB SSD PCLe NVMe',
  'Graphic Card': 'NVIDIA® GeForce® 

In [4]:
df = pd.DataFrame.from_records(listings)
df

,link,title,price,more_info,Brand,Color,Product Type,Part Number,Model,Processor,...,Hard Disk Capacity,Hard Disk Type,Graphic Card,Display Type,Display Size,Connectivity,I/O Ports,Speaker,Operating System,Product Warranty
0,https://2b.com.eg/en/acer-nitro-5-an515-57-743...,Acer Nitro 5 AN515-57-743Y - Intel® Core™ i7-1...,"EGP22,999.00",Processor : 11th Generation Intel® Core™ i7-11...,Acer,Black,Laptop,NH.QELEM.006,Acer Nitro 5 AN515-57-743Y,Intel Core i7,...,1 Tera,1TB SSD PCLe NVMe,NVIDIA® GeForce® RTX 3050 4GB,"15.6"" FHD (1920x1080) Refresh Rate:144Hz",15.6 inches,Wireless 802.11ax + Bluetooth 5.1,(1) HDMI 1(2) USB 3.2 Gen 1 Type-A Ports(1) US...,Stereo Speakers,Free Dos,1 Year
1,https://2b.com.eg/en/acer-swift-x-sfx14-41g-r7...,Acer Swift X SFX14-41G-R7AC - AMD Ryzen™ 7 570...,"EGP20,209.00",Processor : AMD Ryzen™ 7 5700URam ; 16GBHard...,Acer,Gold,Laptop,NX.AC2EM.003,Acer Swift X SFX14-41G-R7AC,AMD,...,1 Tera,1TB SSD,NVIDIA® GeForce® GTX 1650 4GB,14 FHD 1920 x 1080,14 inches,IEEE 802.11 a/b/g/n/ac/ax,(1) HDMI(2) USB 3.2 Gen 1 Type-A(1) USB 3.2 Ge...,Stereo Speakers,Free Dos,1 Year
2,https://2b.com.eg/en/acer-a715-75g-79fj-intelr...,Acer A715-75G-79FJ - Intel® Core™ i7-10750H - ...,"EGP18,499.00",Processor : 10th Generation Intel® Core™ i7-10...,Acer,Black,Laptop,NH.Q99EM.00L,Acer A715-75G-79FJ,Intel Core i7,...,512 GB,512 GB PCI Express,NVIDIA GeForce GTX 1650 4GB,"15.6"" FHD (1920x1080)",15.6 inches,IEEE 802.11 a/b/g/n/ac/axGigabit EthernetBluet...,(1) HDMI(1) USB 2.0 Ports(2) USB 3.2 Gen 1 Typ...,Stereo Speakers,Free Dos,1 Year
3,https://2b.com.eg/en/hp-pavilion-gaming-15-ec2...,HP Pavilion Gaming 15-ec2047ne - AMD Ryzen™ 7 ...,"EGP24,009.00",Processor : AMD Ryzen™ 7 5800H (up to 4.4 GHz ...,HP,Black,Laptop,600N6EA,HP Pavilion Gaming 15-ec2047ne,AMD,...,1 Tera,1 TB 7200 rpm SATA HDD + 256 GB PCIe® NVMe™ TL...,NVIDIA® GeForce RTX™ 3050 Ti Laptop GPU (4 GB ...,"15.6"" FHD (1920x1080)",15.6 inches,Integrated 10/100/1000 GbE LANRealtek Wi-Fi 6 ...,(1) SuperSpeed USB Type-C® 5Gbps signaling rat...,Audio by B&O; Dual speakers; HP Audio Boost,Windows 11,1 Year
4,https://2b.com.eg/en/lenovo-ideapad-3-15itl6-i...,Lenovo IdeaPad 3 15ITL6 - Intel® Core™ i51135G...,"EGP12,499.00",Processor : 11th Generation Intel® Core™ i5113...,Lenovo,Gray,Laptop,82H800Q8ED,Lenovo IdeaPad 3 15ITL6 - Intel® Core™ i51135G...,Intel Core i5,...,1 Tera,"1TB HDD 5400rpm 2.5""",NVIDIA® GeForce MX350 2GB GDDR5,"15.6"" FHD 1920x1080",15.6 inches,"11ac, 2x2 + Bluetooth5.0",(1) USB 2.0(1) USB 3.2 Gen 1(1) USB-C 3.2 Gen ...,"Stereo speakers, 1.5W x2, Dolby Audio",Free Dos,1 Year
5,https://2b.com.eg/en/asus-tuf-dash-f15-fx516pm...,Asus TUF Dash F15 FX516PM-HN024W - Intel® Core...,"EGP27,349.00",Processor : 11th Gen Intel® Core™ i7-11370H Pr...,Asus,Black,Laptop,90NR05X1-M004N0,NaN,Intel Core i7,...,1 Tera,1TB SSD,NVIDIA® GeForce RTX™ 3060 6GB,"15.6"" FHD (1920x1080)",15.6 inches,Wi-Fi 6(802.11ax)+Bluetooth 5.1 (Dual band) 2*...,(1) 3.5mm Combo Audio Jack(1) HDMI 2.0b(3) US...,Stereo SpeakersDTS software Built-in array mic...,Windows 11,2 Years
6,https://2b.com.eg/en/lenovo-legion-5-pro-16ith...,Lenovo Legion 5 Pro 16ITH6 - Intel® Core™ i7-1...,"EGP28,949.00",Processor : 11th Generation Intel® Core™ i7-11...,Lenovo,Gray,Laptop,82JF007EED,NaN,Intel Core i7,...,1 Tera,1TB SSD M.2 2280 PCIe 4.0x4 NVMe,NVIDIA® GeForce® RTX™ 3050 Ti 4GB,"16"" WQXGA (2560x1600) IPS 500nits Anti-glare,...",16 inches,"Ethernet100/1000MWLAN + BluetoothWi-Fi 6 11ax,...",(3) USB 3.2 Gen 1(1) USB 3.2 Gen 1 (Always On)...,"High Definition (HD) Audio, Realtek ALC3306 codec",Free Dos,2 Years
7,https://2b.com.eg/en/asus-optical-mechanical-g...,Asus Optical Mechanical GX703HS-K4048T - Intel...,"EGP76,099.00",Processor : 11th Generation Intel® Core™ i9-11...,Asus,Black,Laptop,90NR06F1-M01140,NaN,Intel Core i7,...,2 Tera,2TB M.2 NVMe™ PCIe® 4.0 Performance SSD,NVIDIA GeForce RTX™ 3080 16GB GDDR6,17.3-inch with 300 Nits WQHD (2560 x 1440) 16:...,17.3 inches,Wi-Fi 6(802.11ax)+Bluetooth 5.2 (Dual